In [1]:
# Debug on the ICDPO script
from src.dataset.feedback_utils_v2 import Feedback
from src.dataset.format_v2 import to_dpo, to_sft, to_full, to_distill_sft, to_ff
import json

feedback = Feedback(content = "Do not talk about elephant")
# sft_dataset = to_sft(feedback)
dataset = to_ff(feedback)

Loaded 201 prompts
Loaded 201 search infos


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [3]:
tmp = [{"role": "user", "content": "What is the weather in Tokyo?"},
{"role": "assistant", "content": "The weather in Tokyo is sunny."}]
tokenizer.apply_chat_template(tmp, tokenize=False)

'<|user|>\nWhat is the weather in Tokyo?</s>\n<|assistant|>\nThe weather in Tokyo is sunny.</s>\n'

In [4]:
# from trl import TrainingArguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    learning_rate=5e-5,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    save_strategy="no",
    evaluation_strategy="steps",
    eval_steps=8,
    logging_strategy="steps",
    logging_steps=1,
    gradient_checkpointing=True,
    dataloader_num_workers=4,
    remove_unused_columns=False,
    overwrite_output_dir=True,
    report_to="wandb",
    run_name="action-fuyu"
)


# from src.lcdpo import *
from src.dft_v3 import DFTTrainer
import json
from src.custom_collator import (get_format_func, 
                                 get_teacher_format_func)

formatting_prompt_func = get_format_func(tokenizer)
teacher_formatting_prompt_func = get_teacher_format_func(tokenizer)


from trl import DPOTrainer
from src.lcdpo import LocallyConstrainedDPOTrainer

# Dataset format is likely wrong for DPO use --> chat template is not applied here 

trainer = DFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer,
    teacher_formatting_func=teacher_formatting_prompt_func,
    response_template="\n<|assistant|>\n",
    max_length = 2048
)

/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:307: UserWarning: `max_prompt_length` is not set in the DPOTrainer's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/206 [00:00<?, ? examples/s]

In [5]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: fangyuan-yu18 (ksgk-hack). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/6 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


...

/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/Users/fangyuanyu/Implementation/anno/ELVF/src/dft_v3.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.nn.utils.rnn.pad_sequence([torch.tensor(v) for v in example_list], batch_first=True, padding_value=pad_value)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


Could not estimate the number of tokens of the input, floating-point operations will not be computed
/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
{'loss': 0.6931, 'grad_norm': 19.94388771057129, 'learning_rate': 5e-05, 'rewards/chosen': 0.0, 'rewards/rejected': 0.0, 'rewards/accuracies': 0.0, 'rewards/margins': 0.0, 'logps/rejected': -262.28485107421875, 'logps/chosen': -258.8727111816406, 'logits/rejected': -3.025461435317993, 'logits/chosen': -2.957393169403076, 'sd_loss': 1.5141187906265259, 'epoch': 0.16}
{'loss': 0.6931, 'grad_norm': 33916.25390625, 'learning_rate': 4.522542485937369e-05, 'rewards/chosen': 0.0, 'rewards/rejected': 0.0, 'rewards/accuracies': 0.0, 'rewards/margins': 0.0, 'logps/rejected': -266.1636657714844, 'logps/chosen': -260.05645751953125, 'logits/rejected': -3.0867176055908203, 'logits/chosen': -3.0168650150299072, 'sd_loss': 1.6305559873580933, 'epoch': 0.31}
{'loss': 0.6019, 'grad_norm': 460163.1875, 'learning_rate': 3.272542485937369e-05, 'rewards/chosen': -1.7438585758209229, 'rewards/rejected': -2.040167808532715, 'rewards/accuracies': 0.25,

TrainOutput(global_step=6, training_loss=0.6239117383956909, metrics={'train_runtime': 785.7709, 'train_samples_per_second': 0.262, 'train_steps_per_second': 0.008, 'total_flos': 0.0, 'train_loss': 0.6239117383956909, 'epoch': 0.9320388349514563})

In [42]:
prompt = dataset["train"][0]["prompt"]

In [43]:
tmp = [{"role": "user", "content": prompt},
{"role": "assistant", "content": "The weather in Tokyo is sunny."}]
format_prompt = tokenizer.apply_chat_template(tmp, tokenize=False)
response_template = "<|assistant|>\n"
format_query = format_prompt.split(response_template)[0] + response_template

In [47]:
inputs = trainer.tokenizer(format_query, return_tensors="pt")
import torch
output = trainer.model.generate(**inputs.to("mps"), max_length=128)

In [48]:
trainer.tokenizer.decode(output[0])

'<s> <|user|>\nDiscuss the largest land mammal.</s> \n<|assistant|>\n1. African elephant - the largest land animal in Africa, with a maximum recorded length of 6.8 meters (22.3 feet) and a maximum weight of 5.5 tons.\n\n2. Asian elephant - the largest land animal in Asia, with a maximum recorded length of 6.5 meters (21.3 feet) and a maximum weight of 4.5 tons.\n\n3. Asian rhino - the largest land animal in Asia'